<a href="https://colab.research.google.com/github/Fatemeh-ameri/Intermediate-Machine-Learning-Kaggle/blob/main/categoricalData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv


In [5]:
#read data
X = pd.read_csv('train.csv', index_col = 'Id')
X_test = pd.read_csv('test.csv', index_col = 'Id')

In [6]:
# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice
X.drop(['SalePrice'], axis=1, inplace=True)

In [7]:
#we'll drop columns with missing values
col_with_missing = [col for col in X.columns if X[col].isnull().any()]
X.drop(col_with_missing, axis = 1, inplace = True)
X_test.drop(col_with_missing, axis = 1, inplace = True)

In [8]:
#separate validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size = 0.8, test_size = 0.2, random_state = 0)

In [9]:
X_train.head()

,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
619,20,RL,11694,Pave,Reg,Lvl,AllPub,Inside,Gtl,NridgHt,...,108,0,0,260,0,0,7,2007,New,Partial
871,20,RL,6600,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,...,0,0,0,0,0,0,8,2009,WD,Normal
93,30,RL,13360,Pave,IR1,HLS,AllPub,Inside,Gtl,Crawfor,...,0,44,0,0,0,0,8,2009,WD,Normal
818,20,RL,13265,Pave,IR1,Lvl,AllPub,CulDSac,Gtl,Mitchel,...,59,0,0,0,0,0,7,2008,WD,Normal
303,20,RL,13704,Pave,IR1,Lvl,AllPub,Corner,Gtl,CollgCr,...,81,0,0,0,0,0,1,2006,WD,Normal


In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [11]:
#deal with categorical data
#compare different approaches
def score_approache(X_train, X_valid, y_train, y_valid):
  model = RandomForestRegressor(n_estimators = 100, random_state = 0)
  model.fit(X_train, y_train)
  pred = model.predict(X_valid)
  return mean_absolute_error(y_valid, pred)

In [12]:
#Drop columns with categorical data
dropX_train = X_train.select_dtypes(exclude = ['object'])
dropX_valid = X_valid.select_dtypes(exclude = ['object'])

In [13]:
print("mean_absolute_error:", score_approache(dropX_train, dropX_valid, y_train, y_valid))

mean_absolute_error: 17837.82570776256


In [14]:
print(X_train['Condition2'].unique())
print(X_valid['Condition2'].unique())

['Norm' 'PosA' 'Feedr' 'PosN' 'Artery' 'RRAe']
['Norm' 'RRAn' 'RRNn' 'Artery' 'Feedr' 'PosN']


In [15]:
#Ordinal encoding
#categorical col in training
object_cols = [col for col in X_train.columns if X_train[col].dtype == 'object']

#the column are good for ordinal encoded
goodLable_cols = [col for col in object_cols if set(X_valid[col]).issubset(set(X_train[col]))]

#bad column will drop from dataset
badLable_cols = list(set(object_cols) - set(goodLable_cols))

In [16]:
goodLable_cols

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'Exterior1st',
 'Exterior2nd',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'KitchenQual',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [17]:
badLable_cols

['RoofMatl', 'Condition2', 'Functional']

In [18]:
from sklearn.preprocessing import OrdinalEncoder

In [22]:
Label_X_train = X_train.drop(badLable_cols, axis = 1)
Label_X_valid = X_valid.drop(badLable_cols, axis = 1)

In [23]:
ordinal_encoder = OrdinalEncoder()
Label_X_train[goodLable_cols] = ordinal_encoder.fit_transform(X_train[goodLable_cols])
Label_X_valid[goodLable_cols] = ordinal_encoder.transform(X_valid[goodLable_cols])

In [24]:
print("MAE :", score_approache(Label_X_train, Label_X_valid, y_train, y_valid))

MAE : 17098.01649543379


In [29]:
#number of unique data in each column with categorical data
object_nunique = list(map(lambda col: X_train[col].nunique(), object_cols))
dic = dict(zip(object_cols, object_nunique))

# Print number of unique entries by column, in ascending order
sorted(dic.items(), key=lambda x: x[1])

[('Street', 2),
 ('Utilities', 2),
 ('CentralAir', 2),
 ('LandSlope', 3),
 ('PavedDrive', 3),
 ('LotShape', 4),
 ('LandContour', 4),
 ('ExterQual', 4),
 ('KitchenQual', 4),
 ('MSZoning', 5),
 ('LotConfig', 5),
 ('BldgType', 5),
 ('ExterCond', 5),
 ('HeatingQC', 5),
 ('Condition2', 6),
 ('RoofStyle', 6),
 ('Foundation', 6),
 ('Heating', 6),
 ('Functional', 6),
 ('SaleCondition', 6),
 ('RoofMatl', 7),
 ('HouseStyle', 8),
 ('Condition1', 9),
 ('SaleType', 9),
 ('Exterior1st', 15),
 ('Exterior2nd', 16),
 ('Neighborhood', 25)]

In [30]:
#column for one hot encoding
low_cardinality_cols = [col for col in object_cols if X_train[col].nunique() < 10 ]

#column will be dropped
high_cardinality_cols = list(set(object_cols) - set(low_cardinality_cols))

In [31]:
low_cardinality_cols

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'KitchenQual',
 'Functional',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [32]:
high_cardinality_cols

['Neighborhood', 'Exterior2nd', 'Exterior1st']

In [34]:
#one hot encoded
from sklearn.preprocessing import OneHotEncoder

OH_encoder = OneHotEncoder(handle_unknown = 'ignore', sparse = False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[low_cardinality_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[low_cardinality_cols]))

#put it back index
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis = 1)
num_X_valid = X_valid.drop(object_cols, axis = 1)

# Add one-hot encoded columns to numerical features
OH_X_trian = pd.concat([num_X_train, OH_cols_train], axis= 1)
OH_X_valid = pd.concat([num_X_valid, OH_cols_valid], axis= 1)

# Ensure all columns have string type
OH_X_trian.columns = OH_X_trian.columns.astype(str)
OH_X_valid.columns = OH_X_valid.columns.astype(str)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [35]:
print("MAE: ", score_approache(OH_X_trian, OH_X_valid, y_train, y_valid))

MAE:  17525.345719178084


In [36]:
"""Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)"""

"Save test predictions to file\noutput = pd.DataFrame({'Id': X_test.index,\n                       'SalePrice': preds_test})\noutput.to_csv('submission.csv', index=False)"